In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import scipy
import matplotlib.pyplot as mlt
import plotly.express as px

# Define Color Pallette

In [2]:
color1 = '#00704A'
color2 = '#FF9FE5'
color3 = '#45062E'
backgroundColor = '#B8A085'
ccs = ['#ACDDA9', '#00704A', '#002F20']
dcs = ['#002619','#008256', '#00de92', '#a1ffdf', '#fdfffe']

# Import Data

In [3]:
df = pd.read_csv('transactions.csv')
df.drop(['Unnamed: 0.1','Unnamed: 0' ], axis =1, inplace=True)
df = df.loc[df['anonymous'] == 0]
df.head()

,person,event,time,offer_id,reward,difficulty,duration,offer_type,web,email,mobile,social,gender,age,became_member_on,income,anonymous,income_cluster
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,bogo,1,1,1,0,F,75,2017-05-09,100000.0,0,0
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,2,10,7,discount,1,1,1,0,M,68,2018-04-26,70000.0,0,1
5,389bc3fa690240e798340f5a15918d5c,offer received,0,f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,bogo,1,1,1,1,M,65,2018-02-09,53000.0,0,1
7,2eeac8d8feae4a8cad5a6af0499a211d,offer received,0,3f207df678b143eea3cee63160fa8bed,0,0,4,informational,1,1,1,0,M,58,2017-11-11,51000.0,0,1
8,aa4862eba776480b8bb9c68455b8c2e1,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,discount,1,1,0,0,F,61,2017-09-11,57000.0,0,1


# Feature Engineering
## We want an indivdual observation for each customer with relevant data

## Offer Count for each Customer   
#### Number of BOGO, Discount, and Information Offers
#### 'offerNumDF'

In [4]:
# Start with getting a offer count for each customer
A = df.groupby(['person', 'offer_type']).count().event.reset_index()
A.columns = ['person', 'offer_type', 'Number of Offers']
A
# Pivot longer
A = A.pivot(columns = 'offer_type', values = 'Number of Offers',index = 'person').reset_index()
A.columns = ['person', 'BOGO_Number', 'Discount_Number', 'Informational_Number']
A.fillna(0, inplace = True)
offerNumDF = A
offerNumDF

,person,BOGO_Number,Discount_Number,Informational_Number
0,0009655768c64bdeb2e877511632db8f,3.0,5.0,4.0
1,0011e0d4e6b944f998e987f904e8c1e5,3.0,6.0,4.0
2,0020c2b971eb4e9188eac86d93036a77,4.0,5.0,2.0
3,0020ccbbb6d84e358d3414a3ff76cffd,6.0,3.0,2.0
4,003d66b6608740288d6cc97a6903f4f0,0.0,8.0,4.0
...,...,...,...,...
14815,fff3ba4757bd42088c044ca26d73817a,3.0,6.0,3.0
14816,fff7576017104bcc8677a8d63322b5e1,6.0,6.0,0.0
14817,fff8957ea8b240a6b5e634b6ee8eafcf,2.0,2.0,1.0
14818,fffad4f4828548d1b5583907f2e9906b,9.0,0.0,2.0


## Aggregate offers channels (web, mobile, social, email) and Reward, difficulty and offer duration
#### 'aggDF'

In [5]:
A = df.loc[:, ['person', 'reward', 'difficulty', 'duration', 'web', 'email', 'social', 'mobile']]
aggDF = A.groupby('person').agg(
    reward_avg = ('reward', 'mean'),
    difficulty_avg = ('difficulty', 'mean'),
    duration_avg = ('duration', 'mean'),
    web = ('web', 'sum'), 
    email = ('email', 'sum'),
    social = ('social', 'sum'),
    mobile = ('mobile', 'sum')
).reset_index()
aggDF

,person,reward_avg,difficulty_avg,duration_avg,web,email,social,mobile
0,0009655768c64bdeb2e877511632db8f,2.083333,5.416667,6.083333,10,12,8,12
1,0011e0d4e6b944f998e987f904e8c1e5,3.000000,7.384615,6.615385,11,13,5,10
2,0020c2b971eb4e9188eac86d93036a77,4.545455,8.181818,7.090909,8,11,11,11
3,0020ccbbb6d84e358d3414a3ff76cffd,3.545455,4.636364,5.727273,9,11,8,11
4,003d66b6608740288d6cc97a6903f4f0,1.833333,8.333333,7.833333,10,12,8,10
...,...,...,...,...,...,...,...,...
14815,fff3ba4757bd42088c044ca26d73817a,2.250000,6.250000,6.750000,9,12,6,12
14816,fff7576017104bcc8677a8d63322b5e1,5.166667,9.166667,8.166667,10,12,10,12
14817,fff8957ea8b240a6b5e634b6ee8eafcf,4.800000,8.000000,6.800000,5,5,4,5
14818,fffad4f4828548d1b5583907f2e9906b,4.090909,4.090909,5.181818,9,11,8,11


## Event aggregation (How many offers received, viewed and completed)
#### 'eventDF'

In [6]:
A = df.loc[:, ['person', 'event', 'web']]
A = A.groupby(['person', 'event']).count().reset_index()
eventDF = A.pivot(index = 'person', columns = 'event', values = 'web').reset_index()
eventDF.fillna(0, inplace = True)
eventDF['ratio_completed'] = round(eventDF['offer completed']/eventDF['offer received'] ,2)
eventDF['ratio_viewed'] = round(eventDF['offer viewed']/ eventDF['offer received'],2)
eventDF

event,person,offer completed,offer received,offer viewed,ratio_completed,ratio_viewed
0,0009655768c64bdeb2e877511632db8f,3.0,5.0,4.0,0.60,0.80
1,0011e0d4e6b944f998e987f904e8c1e5,3.0,5.0,5.0,0.60,1.00
2,0020c2b971eb4e9188eac86d93036a77,3.0,5.0,3.0,0.60,0.60
3,0020ccbbb6d84e358d3414a3ff76cffd,3.0,4.0,4.0,0.75,1.00
4,003d66b6608740288d6cc97a6903f4f0,3.0,5.0,4.0,0.60,0.80
...,...,...,...,...,...,...
14815,fff3ba4757bd42088c044ca26d73817a,3.0,6.0,3.0,0.50,0.50
14816,fff7576017104bcc8677a8d63322b5e1,3.0,5.0,4.0,0.60,0.80
14817,fff8957ea8b240a6b5e634b6ee8eafcf,0.0,3.0,2.0,0.00,0.67
14818,fffad4f4828548d1b5583907f2e9906b,3.0,4.0,4.0,0.75,1.00


## Create Demographics DF (age, income, year been a  member, gender)
#### 'demoDF'

In [7]:
demoDF = df.loc[:,['person', 'gender', 'age', 'became_member_on', 'income']]
demoDF = demoDF.groupby('person').agg(
    gender = ('gender', 'first'),
    age = ('age', 'first'),
    income = ('income', 'first'),
    became_member_on = ('became_member_on', 'first')
).reset_index()
# convert became a member on to date
demoDF['became_member_on'] = pd.to_datetime(demoDF['became_member_on'])
demoDF['year_became_member'] = demoDF['became_member_on'].dt.year
demoDF.drop(['became_member_on'], axis = 1, inplace = True)
demoDF

,person,gender,age,income,year_became_member
0,0009655768c64bdeb2e877511632db8f,M,33,72000.0,2017
1,0011e0d4e6b944f998e987f904e8c1e5,O,40,57000.0,2018
2,0020c2b971eb4e9188eac86d93036a77,F,59,90000.0,2016
3,0020ccbbb6d84e358d3414a3ff76cffd,F,24,60000.0,2016
4,003d66b6608740288d6cc97a6903f4f0,F,26,73000.0,2017
...,...,...,...,...,...
14815,fff3ba4757bd42088c044ca26d73817a,F,69,83000.0,2015
14816,fff7576017104bcc8677a8d63322b5e1,M,71,73000.0,2017
14817,fff8957ea8b240a6b5e634b6ee8eafcf,M,71,56000.0,2018
14818,fffad4f4828548d1b5583907f2e9906b,M,34,34000.0,2017


## Attach how much money they spent and most recent transaction date

In [8]:
money = pd.read_csv('transcriptT.csv')
money.drop(['Unnamed: 0', 'value'], axis = 1, inplace = True)
money = money.groupby('person').agg(
    most_recent = ('time', 'last'),
    total_spent = ('spent', 'sum'),
    transactions_number = ('event', 'count')
).reset_index()
money

,person,most_recent,total_spent,transactions_number
0,0009655768c64bdeb2e877511632db8f,696,127.60,8
1,00116118485d4dfda04fdbaba9a87b5c,474,4.09,3
2,0011e0d4e6b944f998e987f904e8c1e5,654,79.46,5
3,0020c2b971eb4e9188eac86d93036a77,708,196.86,8
4,0020ccbbb6d84e358d3414a3ff76cffd,672,154.05,12
...,...,...,...,...
16573,fff3ba4757bd42088c044ca26d73817a,552,580.98,11
16574,fff7576017104bcc8677a8d63322b5e1,696,29.94,6
16575,fff8957ea8b240a6b5e634b6ee8eafcf,576,12.15,5
16576,fffad4f4828548d1b5583907f2e9906b,678,88.83,12


## Merge

In [9]:
mergeDF = offerNumDF.merge(aggDF, how = 'left', on = 'person')
mergeDF = mergeDF.merge(eventDF, how = 'left', on = 'person')

mergeDF = demoDF.merge(mergeDF, how = 'left', on = 'person')
mergeDF = mergeDF.merge(money, how = 'left', on = 'person')
mergeDF

,person,gender,age,income,year_became_member,BOGO_Number,Discount_Number,Informational_Number,reward_avg,difficulty_avg,...,social,mobile,offer completed,offer received,offer viewed,ratio_completed,ratio_viewed,most_recent,total_spent,transactions_number
0,0009655768c64bdeb2e877511632db8f,M,33,72000.0,2017,3.0,5.0,4.0,2.083333,5.416667,...,8,12,3.0,5.0,4.0,0.60,0.80,696.0,127.60,8.0
1,0011e0d4e6b944f998e987f904e8c1e5,O,40,57000.0,2018,3.0,6.0,4.0,3.000000,7.384615,...,5,10,3.0,5.0,5.0,0.60,1.00,654.0,79.46,5.0
2,0020c2b971eb4e9188eac86d93036a77,F,59,90000.0,2016,4.0,5.0,2.0,4.545455,8.181818,...,11,11,3.0,5.0,3.0,0.60,0.60,708.0,196.86,8.0
3,0020ccbbb6d84e358d3414a3ff76cffd,F,24,60000.0,2016,6.0,3.0,2.0,3.545455,4.636364,...,8,11,3.0,4.0,4.0,0.75,1.00,672.0,154.05,12.0
4,003d66b6608740288d6cc97a6903f4f0,F,26,73000.0,2017,0.0,8.0,4.0,1.833333,8.333333,...,8,10,3.0,5.0,4.0,0.60,0.80,696.0,48.34,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14815,fff3ba4757bd42088c044ca26d73817a,F,69,83000.0,2015,3.0,6.0,3.0,2.250000,6.250000,...,6,12,3.0,6.0,3.0,0.50,0.50,552.0,580.98,11.0
14816,fff7576017104bcc8677a8d63322b5e1,M,71,73000.0,2017,6.0,6.0,0.0,5.166667,9.166667,...,10,12,3.0,5.0,4.0,0.60,0.80,696.0,29.94,6.0
14817,fff8957ea8b240a6b5e634b6ee8eafcf,M,71,56000.0,2018,2.0,2.0,1.0,4.800000,8.000000,...,4,5,0.0,3.0,2.0,0.00,0.67,576.0,12.15,5.0
14818,fffad4f4828548d1b5583907f2e9906b,M,34,34000.0,2017,9.0,0.0,2.0,4.090909,4.090909,...,8,11,3.0,4.0,4.0,0.75,1.00,678.0,88.83,12.0


### Look for NAs

In [10]:
mergeDF.isna().any()

person                  False
gender                  False
age                     False
income                  False
year_became_member      False
BOGO_Number             False
Discount_Number         False
Informational_Number    False
reward_avg              False
difficulty_avg          False
duration_avg            False
web                     False
email                   False
social                  False
mobile                  False
offer completed         False
offer received          False
offer viewed            False
ratio_completed         False
ratio_viewed            False
most_recent              True
total_spent              True
transactions_number      True
dtype: bool

### Only NA values are in actual transactions, meaning they never bought anything, fill nas with 0

In [11]:
mergeDF.fillna(0, inplace=True)
mergeDF.isna().any()

person                  False
gender                  False
age                     False
income                  False
year_became_member      False
BOGO_Number             False
Discount_Number         False
Informational_Number    False
reward_avg              False
difficulty_avg          False
duration_avg            False
web                     False
email                   False
social                  False
mobile                  False
offer completed         False
offer received          False
offer viewed            False
ratio_completed         False
ratio_viewed            False
most_recent             False
total_spent             False
transactions_number     False
dtype: bool

## Calculate RFM score (recency, frequency, monetary)

In [12]:
mergeDF['M_score'] = pd.cut(mergeDF['total_spent'], bins=[-1,
       np.percentile(mergeDF['total_spent'], 20),
       np.percentile(mergeDF['total_spent'], 40),
       np.percentile(mergeDF['total_spent'], 60),
       np.percentile(mergeDF['total_spent'], 80),
       mergeDF['total_spent'].max()],
                           labels = [1,2,3,4,5]).astype('int')

In [13]:
mergeDF[['M_score', 'total_spent']]

,M_score,total_spent
0,4,127.60
1,3,79.46
2,5,196.86
3,4,154.05
4,2,48.34
...,...,...
14815,5,580.98
14816,2,29.94
14817,1,12.15
14818,3,88.83


In [14]:
mergeDF['R_score'] = pd.cut(mergeDF['most_recent'], bins=[-1,
       np.percentile(mergeDF['most_recent'], 20),
       np.percentile(mergeDF['most_recent'], 40),
       np.percentile(mergeDF['most_recent'], 60),
       np.percentile(mergeDF['most_recent'], 80),
       mergeDF['most_recent'].max()],
                           labels = [1,2,3,4,5]).astype('int')

In [15]:
mergeDF[['R_score', 'most_recent']]

,R_score,most_recent
0,4,696.0
1,3,654.0
2,5,708.0
3,4,672.0
4,4,696.0
...,...,...
14815,1,552.0
14816,4,696.0
14817,2,576.0
14818,4,678.0


In [16]:
mergeDF['F_score'] = pd.cut(mergeDF['transactions_number'], bins=[-1,
       np.percentile(mergeDF['transactions_number'], 20),
       np.percentile(mergeDF['transactions_number'], 40),
       np.percentile(mergeDF['transactions_number'], 60),
       np.percentile(mergeDF['transactions_number'], 80),
       mergeDF['transactions_number'].max()],
                           labels = [1,2,3,4,5]).astype('int')

In [17]:
mergeDF[['F_score', 'transactions_number']]

,F_score,transactions_number
0,3,8.0
1,2,5.0
2,3,8.0
3,4,12.0
4,5,18.0
...,...,...
14815,4,11.0
14816,2,6.0
14817,2,5.0
14818,4,12.0


In [18]:
mergeDF

,person,gender,age,income,year_became_member,BOGO_Number,Discount_Number,Informational_Number,reward_avg,difficulty_avg,...,offer received,offer viewed,ratio_completed,ratio_viewed,most_recent,total_spent,transactions_number,M_score,R_score,F_score
0,0009655768c64bdeb2e877511632db8f,M,33,72000.0,2017,3.0,5.0,4.0,2.083333,5.416667,...,5.0,4.0,0.60,0.80,696.0,127.60,8.0,4,4,3
1,0011e0d4e6b944f998e987f904e8c1e5,O,40,57000.0,2018,3.0,6.0,4.0,3.000000,7.384615,...,5.0,5.0,0.60,1.00,654.0,79.46,5.0,3,3,2
2,0020c2b971eb4e9188eac86d93036a77,F,59,90000.0,2016,4.0,5.0,2.0,4.545455,8.181818,...,5.0,3.0,0.60,0.60,708.0,196.86,8.0,5,5,3
3,0020ccbbb6d84e358d3414a3ff76cffd,F,24,60000.0,2016,6.0,3.0,2.0,3.545455,4.636364,...,4.0,4.0,0.75,1.00,672.0,154.05,12.0,4,4,4
4,003d66b6608740288d6cc97a6903f4f0,F,26,73000.0,2017,0.0,8.0,4.0,1.833333,8.333333,...,5.0,4.0,0.60,0.80,696.0,48.34,18.0,2,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14815,fff3ba4757bd42088c044ca26d73817a,F,69,83000.0,2015,3.0,6.0,3.0,2.250000,6.250000,...,6.0,3.0,0.50,0.50,552.0,580.98,11.0,5,1,4
14816,fff7576017104bcc8677a8d63322b5e1,M,71,73000.0,2017,6.0,6.0,0.0,5.166667,9.166667,...,5.0,4.0,0.60,0.80,696.0,29.94,6.0,2,4,2
14817,fff8957ea8b240a6b5e634b6ee8eafcf,M,71,56000.0,2018,2.0,2.0,1.0,4.800000,8.000000,...,3.0,2.0,0.00,0.67,576.0,12.15,5.0,1,2,2
14818,fffad4f4828548d1b5583907f2e9906b,M,34,34000.0,2017,9.0,0.0,2.0,4.090909,4.090909,...,4.0,4.0,0.75,1.00,678.0,88.83,12.0,3,4,4
